In [271]:
import numpy as np

data = np.load('my_mnist_model2.npz') #Laste ned trente parametre fra det nevrale netverket

# Rebuild layers list - exactly as your model expects
layers = [
    (data['W0'], data['b0']),
    (data['W1'], data['b1']),
    (data['W2'], data['b2']),
    (data['W3'], data['b3'])
]

print("Model loaded!")
#print(layers)

Model loaded!


In [272]:
# MY OWN DIGIT
import numpy as np
from PIL import Image

B = "22.png"  #Åpne bildet 28 x 28 piksler laget i paint (Bildeegenskaper sett 28 x 28 piksler)
img = Image.open(B).convert('L')           # 'L' = grayscale
img = img.resize((28, 28), Image.Resampling.NEAREST)  # or LANCZOS for smoother

arr = np.array(img)                           # shape (28,28), values 0-255
for i in range(28):
    for j in range(28):
        arr[i,j] = round(arr[i,j])            # classic Python round()
        arr[i,j] = 255-arr[i,j]

    # Decide inversion based on your drawing style:
    # Case 1: white paper + dark ink  → NO inversion

#print(arr)
arr = arr / 255.0

"""
for row in arr:
    print(' '.join(f"{x:2}" for x in row))
    # Case 2: black background + light digit → invert
    # arr = (255 - arr) / 255.0

    # Optional: slight threshold / contrast boost (helps a lot!)
    # arr = np.maximum(arr - 0.05, 0) ** 1.3     # remove very light noise + increase contrast
"""

# Your 28×28 array (can be int or float)
arr_2d = np.array(arr)          # shape (28, 28)

# Flatten it
vector = arr_2d.flatten()       # or arr_2d.ravel()

print(vector.shape)             # → (784,)





(784,)


In [273]:
import autograd.numpy as np  
from autograd import grad, elementwise_grad
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

def ReLU(z):
    return np.where(z > 0, z, 0)

def softmax(z):
    """Compute softmax values for each set of scores in the rows of the matrix z.
    Used with batched input data."""
    e_z = np.exp(z - np.max(z, axis=1)[:, np.newaxis]) #substract max per row, avoids instability
    return e_z / np.sum(e_z, axis=1)[:, np.newaxis]

def feed_forward(input, layers, activation_funcs):
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        z = a @ W + b
        a = activation_func(z)
    return a

def accuracy(predictions, targets):
    one_hot_predictions = np.zeros(predictions.shape)
    for i, prediction in enumerate(predictions):
        one_hot_predictions[i, np.argmax(prediction)] = 1
    return accuracy_score(one_hot_predictions, targets)

def cross_entropy(predict, target):
    return np.mean(-np.sum(target * np.log(predict + 1e-10), axis=1))



In [274]:
#MY DIGIT
network_input_size = 784
layer_output_sizes = [512, 256, 128, 10]  
activation_funcs = [ReLU, ReLU, ReLU, softmax]

my_input = vector.reshape(1, -1)


y_label = 9                     #Endre target for hvert tall manuelt! HER
ymin = np.eye(10)[y_label]

print(ymin)
#print(my_input)

prediction = feed_forward(my_input, layers, activation_funcs)   

predicted_class = np.argmax(prediction)
confidence = prediction[0, predicted_class] * 100

print(cross_entropy(prediction, ymin))
print(prediction)
print(predicted_class)
print(confidence)

from IPython.display import Image, display
# Option B - with some control
display(Image(filename=B, width=280, height=280))



######
def print_digit(vector, threshold=0.1):
    img = vector.reshape(28, 28)
    # Characters from light → dark
    chars = ' .:-=+*#%@'
    for row in img:
        line = ''.join(chars[int(val * (len(chars)-1))] for val in row)
        print(line)

#print_digit(my_input)
####


[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
19.768577082029186
[[5.64519956e-08 1.17757410e-07 9.99972826e-01 2.79200017e-06
  1.23827166e-09 6.83918242e-10 8.28489770e-08 9.28948321e-06
  1.48312303e-05 2.49786189e-09]]
2
99.99728258078471


In [275]:
#MNIST DIGIT

#Applying FFNN base above to classification problem, MNIST:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

#Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')

#Extract data
X = mnist.data

D = 4               #ENDRE HER

y_labels = mnist.target.astype(int)
X = X / 255.0

print(y_labels[D])
#One-hot encode labels
y = np.eye(10)[y_labels] #Convert to integers
print(y[D])
#print(X[D])

network_input_size = 784
layer_output_sizes = [512, 256, 128, 10]  # output 10 classes
activation_funcs = [ReLU, ReLU, ReLU, softmax]

my_input = X[D].reshape(1, -1)

#print(my_input)

prediction = feed_forward(my_input, layers, activation_funcs)   # shape (1, 10)

predicted_class = np.argmax(prediction)
confidence = prediction[0, predicted_class] * 100

predicted_class = np.argmax(prediction)
confidence = prediction[0, predicted_class] * 100

print(cross_entropy(prediction, y[D]))
print(prediction)
print(predicted_class)
print(confidence)

print_digit(X[D])           # your 784-element array / list

9
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
8.638809470728372e-05
[[1.19399059e-08 4.81688269e-10 1.54754176e-08 1.64892834e-05
  1.39112734e-05 9.70589083e-06 8.11416665e-10 6.42135727e-06
  3.98279500e-05 9.99913616e-01]]
9
99.99136155366367
                            
                            
                            
                            
                            
                            
                            
            .+#%%--+.       
           -%%%*#%%%+       
          :%%*:  *%%=       
         -%%*   -%%#        
        =%%+    #%%:        
       =%%*    :%%=         
       %%*   .#%%+          
       %%. =*%%%%:          
       %%%%%%#%%%           
       .%%#=. *%+           
             :@%-           
             :%%            
              %%            
             :%%            
             -%%            
             .@%            
              #%.           
              -%*.          
               *%*          
            